In [1]:
import sys
import BAC0
import time
import pandas as pd
import os

# bacnet = BAC0.connect(ip='192.168.1.51/24')
interval = 5

def save(path, df, sheet):
    if not os.path.isfile(path):
        pd.DataFrame().to_excel(path, sheet_name=sheet)

    with pd.ExcelWriter(path, engine="openpyxl", mode="a") as writer:
        workBook = writer.book
        try:
            workBook.remove(workBook[sheet])
        except:
            print("Worksheet does not exist")
        finally:
            df.to_excel(writer, sheet_name=sheet, index=False)


# File for raw data #
cwd = os.path.dirname(os.path.realpath(sys.argv[0]))
raw = pd.read_excel(cwd + '/source/raw.xlsx', sheet_name='raw')
points = pd.read_excel(cwd + '/source/raw.xlsx', sheet_name='points', index_col='BACnet Object Name')
blocks = pd.read_excel(cwd + '/source/raw.xlsx', sheet_name='blocks', index_col='Block No.')
raw.columns = raw.columns.str.replace(' ', '_')

#get all devices
bacnet = BAC0.connect(ip='192.168.1.51/24')
bacnet.whois()
devices = pd.DataFrame(bacnet.devices)

# Reading properties from device #
df = pd.DataFrame(columns=['Shown Name', 'Value', 'Unit', 'Block', 'Page'])
df.index.name = 'Object Name'

for row in raw.itertuples():
    for device in devices.itertuples(index=False):
        if row.Device_Name == device[0]:
            read = str(device[2]) + ' ' + row.BACnet_Object_type + ' ' + str(int(row.BACnet_Object_Instance)) + ' presentValue'
            block, point = row.BACnet_Object_Name.split('-', 1)
            if point in points.index:
                df.loc[row.BACnet_Object_Name] = [
                    points['Shown Name'][point],
                    "{:.1f}".format(bacnet.read(read)),
                    points['Unit'][point],
                    block,
                    points['Page'][point],
                ]
            else:
                df.loc[row.BACnet_Object_Name] = ['point not found', "{:.1f}".format(bacnet.read(read)), 'point not found', block, 'point not found']
            break

print(df, "\n")
for code, df_block in df.groupby('Block'):
    if code in blocks.index:
        save(cwd + '/source/data.xlsx', df_block.loc[:, df_block.columns!='Block'], blocks['Name'][code])
    else:
        save(cwd + '/source/data.xlsx', df_block.loc[:, df_block.columns!='Block'], blocks['Name']['B01'])

bacnet.disconnect()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python39\\site-packages/source/raw.xlsx'